\title{Satellite Imagery Search and Sort} \author{} \maketitle

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#The-Data-source" data-toc-modified-id="The-Data-source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The Data source</a></div><div class="lev1 toc-item"><a href="#Other-requeired-tool" data-toc-modified-id="Other-requeired-tool-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Other requeired tool</a></div><div class="lev1 toc-item"><a href="#The-codes" data-toc-modified-id="The-codes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The codes</a></div><div class="lev2 toc-item"><a href="#get-the-data" data-toc-modified-id="get-the-data-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>get the data</a></div><div class="lev2 toc-item"><a href="#Apply-fmask-algorthm" data-toc-modified-id="Apply-fmask-algorthm-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Apply fmask algorthm</a></div><div class="lev2 toc-item"><a href="#detecting-lowest-cloud-cover-imagery" data-toc-modified-id="detecting-lowest-cloud-cover-imagery-33"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>detecting lowest cloud cover imagery</a></div>

# The Data source

1. NASA's Landsat satellite imagery

# Other requeired tool
1. Anaconda Python distributuion
1. Jupyter notebook
1. Google aearth
1. google cloud platform
1. Qgis

# The codes

## get the data

In [ ]:
import os
import pandas as pd

pathrow2=pd.read_csv('landsat4apna.csv')

cities=['Coimbatore']



for city in cities:
    pathrow3=pathrow2[pathrow2.city==city]
    newpath = '/home/sunbird/landsat/'+city 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for index, row in pathrow3.iterrows():
        os.popen("gsutil -m cp -r "+row['BASE_URL']+" /home/sunbird/landsat/"+city+"/").readlines()

## Apply fmask algorthm

In [ ]:
import os
import sys
import argparse

from fmask import landsatangles
from fmask import config

from rios import fileinfo
import numpy
import glob

parentfolder='/home/sunbird/landsat/coimbatore'



os.chdir(parentfolder)
lcfolders=glob.glob('L*')

from pyproj import Proj, transform
import datetime

def sunAnglesForExtent(imgInfo, mtlInfo):
    """
    Return array of sun azimuth and zenith for each of the corners of the image
    extent. Note that this is the raster extent, not the corners of the swathe.

    The algorithm used here has been copied from the 6S possol() subroutine. The
    Fortran code I copied it from was .... up to the usual standard in 6S. So, the
    notation is not always clear.

    """
    inProj = Proj(init='epsg:32643')
    outProj = Proj(init='epsg:4326')
    cornerLatLong = imgInfo.getCorners()
    (ul_long, ul_lat, ur_long, ur_lat, lr_long, lr_lat, ll_long, ll_lat) = cornerLatLong
    ul_long1, ul_lat1=transform(inProj,outProj,ul_long, ul_lat)
    ur_long1, ur_lat1=transform(inProj,outProj,ur_long, ur_lat)
    lr_long1, lr_lat1=transform(inProj,outProj,lr_long, lr_lat,)
    ll_long1, ll_lat1=transform(inProj,outProj,ll_long, ll_lat)
    pts = numpy.array([
        [ul_long1, ul_lat1],
        [ur_long1, ur_lat1],
        [ll_long1, ll_lat1],
        [lr_long1, lr_lat1]
    ])
    longDeg = pts[:, 0]
    latDeg = pts[:, 1]
    # Date/time in UTC
    dateStr = mtlInfo['DATE_ACQUIRED']
    timeStr = mtlInfo['SCENE_CENTER_TIME'].replace('Z', '')
    ymd = [int(i) for i in dateStr.split('-')]
    dateObj = datetime.date(ymd[0], ymd[1], ymd[2])
    julianDay = (dateObj - datetime.date(ymd[0], 1, 1)).days + 1
    juldayYearEnd = (datetime.date(ymd[0], 12, 31) - datetime.date(ymd[0], 1, 1)).days + 1
    # Julian day as a proportion of the year
    jdp = julianDay / juldayYearEnd
    # Hour in UTC
    hms = [float(x) for x in timeStr.split(':')]
    hourGMT = hms[0] + hms[1] / 60.0 + hms[2] / 3600.0
    (sunAz, sunZen) = landsatangles.sunAnglesForPoints(latDeg, longDeg, hourGMT, jdp)
    sunAngles = numpy.vstack((sunAz, sunZen)).T
    return sunAngles


def getCtrLatLong1(imgInfo):
    """
    Return the lat/long of the centre of the image as
        (ctrLat, ctrLong)

    """
    cornerLatLong = imgInfo.getCorners()
    inProj = Proj(init='epsg:32643')
    outProj = Proj(init='epsg:4326')
    (ul_long, ul_lat, ur_long, ur_lat, lr_long, lr_lat, ll_long, ll_lat) = cornerLatLong
    ul_long1, ul_lat1=transform(inProj,outProj,ul_long, ul_lat)
    ur_long1, ur_lat1=transform(inProj,outProj,ur_long, ur_lat)
    lr_long1, lr_lat1=transform(inProj,outProj,lr_long, lr_lat)
    ll_long1, ll_lat1=transform(inProj,outProj,ll_long, ll_lat)
    ctrLat = numpy.array([ul_lat, ur_lat, lr_lat, ll_lat]).mean()
    ctrLong = numpy.array([ul_long, ur_long, lr_long, ll_long]).mean()
    return (ctrLat, ctrLong)



def makeAnglesImage(templateimg, outfile, nadirLine, extentSunAngles, satAzimuth, imgInfo):
    """
    Make a single output image file of the sun and satellite angles for every
    pixel in the template image.

    """
    imgInfo  = fileinfo.ImageInfo(templateimg)
    infiles = applier.FilenameAssociations()
    outfiles = applier.FilenameAssociations()
    otherargs = applier.OtherInputs()
    controls = applier.ApplierControls()
    infiles.img = templateimg
    outfiles.angles = outfile
    (ctrLat, ctrLong) = getCtrLatLong1(imgInfo)
    otherargs.R = landsatangles.localRadius(ctrLat)
    otherargs.nadirLine = nadirLine
    otherargs.xMin = imgInfo.xMin
    otherargs.xMax = imgInfo.xMax
    otherargs.yMin = imgInfo.yMin
    otherargs.yMax = imgInfo.yMax
    otherargs.extentSunAngles = extentSunAngles
    otherargs.satAltitude = 705000      # Landsat nominal altitude in metres
    otherargs.satAzimuth = satAzimuth
    otherargs.radianScale = 100        # Store pixel values as (radians * radianScale)
    controls.setStatsIgnore(500)
    applier.apply(landsatangles.makeAngles, infiles, outfiles, otherargs, controls=controls)
    
    
for lcfolder in lcfolders:
    os.chdir(parentfolder)
    os.chdir(lcfolder+'/')
    os.popen("gdal_merge.py -separate -o "+'ref_'+lcfolder+".tif"+" apna*_B[1-7,9].tif").readlines()
    os.popen("gdal_merge.py -separate -o "+'thermal_'+lcfolder+".tif"+" apna*_B1[0,1].tif").readlines()
    mtlInfo = config.readMTLFile(lcfolder+'_MTL.txt')
    imgInfo = fileinfo.ImageInfo('ref_'+lcfolder+".tif")
    corners = landsatangles.findImgCorners('ref_'+lcfolder+".tif", imgInfo)
    nadirLine = landsatangles.findNadirLine(corners)
    extentSunAngles = sunAnglesForExtent(imgInfo, mtlInfo)
    satAzimuth = landsatangles.satAzLeftRight(nadirLine)
    makeAnglesImage('ref_'+lcfolder+".tif", 'angles_'+lcfolder+".tif",nadirLine, extentSunAngles, satAzimuth, imgInfo)
    os.popen("/home/sunbird/landsat/coimbatore/fmask_usgsLandsatSaturationMask.py -i "+"ref_"+lcfolder+".tif"+" -m *_MTL.txt -o "+"saturationmask_"+lcfolder+".tif").readlines()
    os.popen("/home/sunbird/landsat/coimbatore/fmask_usgsLandsatTOA.py -i "+"ref_"+lcfolder+".tif"+" -m *_MTL.txt -z angles_"+lcfolder+".tif "+"-o toa_"+lcfolder+".tif").readlines()
    os.popen("/home/sunbird/landsat/coimbatore/fmask_usgsLandsatStacked.py -t "+"thermal_"+lcfolder+".tif"+" -a "+"toa_"+lcfolder+".tif"+" -m *_MTL.txt -z angles_"+lcfolder+".tif"+" -s "+"saturationmask_"+lcfolder+".tif"+" -o "+"cloud_"+lcfolder+".tif").readlines()
    os.chdir(parentfolder)
    print "a lcfolder done"
    
#os.listdir('.')

## detecting lowest cloud cover imagery

In [ ]:
import os
import glob
import rasterio


parentfolder='/home/sunbird/landsat/coimbatore'



os.chdir(parentfolder)
lcfolders=glob.glob('L*')

for lcfolder in lcfolders:
    os.chdir(parentfolder)
    os.chdir(lcfolder+'/')
    dataset = rasterio.open("cloud_"+lcfolder+".tif")
    data=dataset.read(1)
    unique, counts = np.unique(data, return_counts=True)
    dattif=dict(zip(unique, counts))
    print lcfolder, dattif
    dattif1={x: dattif[x] for x in [2,3] if x in dattif}
    percloud=round(((sum([i for i in dattif1.values()])*30*30)/(1000*1000)) / ((sum([i for i in dattif.values()])*30*30)/(1000*1000)) *100,1)
    print lcfolder, percloud

In [1]:
import rasterio

In [3]:
dataset = rasterio.open("/home/sunbird/Desktop/apna_LC81440522015013LGN00_B2.tif")
data=dataset.read(1)

In [4]:
data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [7]:
data.shape

(1857, 1834)